# Sidesh's 2D to 3D Reconstruction Implementation

This notebook contains Sidesh's custom implementation for 2D to 3D reconstruction with advanced depth processing.

In [ ]:
# Import libraries
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d

## Load MiDaS Depth Model

In [ ]:
# Load MiDaS model
model_type = 'DPT_Large'
midas = torch.hub.load('intel-isl/MiDaS', model_type)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
midas.to(device)
midas.eval()

# Load transforms
midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')

if model_type == 'DPT_Large' or model_type == 'DPT_Hybrid':
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

print(f'Model loaded on {device}')

## Process Input Image

In [ ]:
# Load image
img_path = 'input.jpg'
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display
plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Input Image')
plt.axis('off')
plt.show()

## Generate Depth Map

In [ ]:
# Transform and predict
input_batch = transform(img_rgb).to(device)

with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img_rgb.shape[:2],
        mode='bicubic',
        align_corners=False,
    ).squeeze()

depth = prediction.cpu().numpy()

# Normalize depth
depth_normalized = (depth - depth.min()) / (depth.max() - depth.min())
depth_normalized = (depth_normalized * 255).astype(np.uint8)

# Save depth map
cv2.imwrite('car_depth_output.jpg', depth_normalized)
print('Depth map saved')

In [ ]:
# Visualize depth
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(depth, cmap='plasma')
plt.title('Depth Map')
plt.colorbar()
plt.axis('off')
plt.show()

## Convert to 3D Point Cloud

In [ ]:
def create_point_cloud(depth_map, rgb_image, focal=500):
    h, w = depth_map.shape
    
    # Generate grid
    y, x = np.mgrid[0:h, 0:w]
    
    # Calculate 3D points
    z = depth_map
    x = (x - w/2) * z / focal
    y = (y - h/2) * z / focal
    
    points = np.stack([x, y, z], axis=-1).reshape(-1, 3)
    colors = rgb_image.reshape(-1, 3) / 255.0
    
    return points, colors

points_3d, colors_3d = create_point_cloud(depth, img_rgb)
print(f'Generated {len(points_3d)} 3D points')

In [ ]:
# Create Open3D point cloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points_3d)
point_cloud.colors = o3d.utility.Vector3dVector(colors_3d)

# Statistical outlier removal
cl, ind = point_cloud.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
point_cloud_filtered = point_cloud.select_by_index(ind)

# Save
o3d.io.write_point_cloud('3d_keypoints.ply', point_cloud_filtered)
print(f'Saved point cloud with {len(point_cloud_filtered.points)} points')

## Visualize 3D Point Cloud

In [ ]:
# Visualize
o3d.visualization.draw_geometries(
    [point_cloud_filtered],
    window_name='3D Reconstruction',
    width=1024,
    height=768,
    point_show_normal=False
)

## Advanced: Mesh Reconstruction

In [ ]:
# Compute normals
point_cloud_filtered.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

# Poisson reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    point_cloud_filtered, depth=9
)

# Remove low density vertices
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Save mesh
o3d.io.write_triangle_mesh('reconstructed_3d_mesh.ply', mesh)
print(f'Mesh: {len(mesh.vertices)} vertices, {len(mesh.triangles)} triangles')

# Visualize mesh
o3d.visualization.draw_geometries([mesh], window_name='3D Mesh')